# LAB 1: Path Search for the Set Covering Problem

In [163]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [164]:
PROBLEM_SIZE = 25
NUM_SETS = 50
SETS = tuple(
    np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)
)
State = namedtuple("State", ["taken", "not_taken"])

In [165]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [166]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

## Greedy Best First

In [167]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [168]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

5it [00:00, 1965.47it/s]

Solved in 5 steps (5 tiles)


In [169]:
current_state

State(taken={6, 12, 25, 27, 28}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49})

In [170]:
goal_check(current_state)

True

## A*

### Algorithm

In [171]:
def f(state, cost, heuristic):
    return cost(state) + heuristic(state)


def a_star(cost, heuristic):
    frontier = PriorityQueue()

    state = State(set(), set(range(NUM_SETS)))
    frontier.put((f(state, cost, heuristic), state))

    counter = 0
    _, current_state = frontier.get()
    with tqdm(total=None) as pbar:
        while not goal_check(current_state):
            counter += 1
            for action in current_state[1]:
                new_state = State(
                    current_state.taken ^ {action},
                    current_state.not_taken ^ {action},
                )
                frontier.put((f(new_state, cost, heuristic), new_state))
            _, current_state = frontier.get()
            pbar.update(1)

    print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
    return current_state

### Heuristics defined in the lecture

- `h1(state)`: calculates the number of additional sets required to cover the remaining elements, assuming the size of the largest set;
- `h2(state)`: similar to `h1(state)`, but does not consider sets that had already been taken when finding the largest set;
- `h3(state)`: similar to `h2(state)` and `h(3)`, but it estimates the number of additional needed sets by iteratively selecting sets in descending order of size until the uncovered elements are sufficiently covered; 

In [172]:
def cost(state):
    return len(state.taken)


def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(
        sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS
    )
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted(
        (sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS),
        reverse=True,
    )
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

#### Results

In [173]:
print("Heuristic 1")
print(a_star(cost, h1))
print()

print("Heuristic 2")
print(a_star(cost, h2))
print()

print("Heuristic 3")
print(a_star(cost, h3))

Heuristic 1


1572it [00:11, 142.32it/s]


Solved in 1,572 steps (4 tiles)
State(taken={25, 23, 9, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49})

Heuristic 2


675it [00:06, 112.26it/s]


Solved in 675 steps (4 tiles)
State(taken={9, 3, 44, 39}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 45, 46, 47, 48, 49})

Heuristic 3


173it [00:01, 114.20it/s]

Solved in 173 steps (4 tiles)
State(taken={9, 3, 12, 39}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49})


### Additional heuristics

- At first, guided from what we had seen in the lecture, I attempted to use the greedy algorithm's distance function as the heuristic, but that is not correct for A* since it is not asmissible, and will not find the optimal solution.
- I include it here along with a function that verifies if a given heuristic is admissible by trying to find states where `h(s)` is not less than or equal to the actual cost to reach the goal from `s`.

In [180]:
def h4(state):
    return PROBLEM_SIZE - np.count_nonzero(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([0] * PROBLEM_SIZE),
        )
    )

print("Heuristic 4")
print(a_star(cost, h4))

Heuristic 4


6it [00:00, 1934.05it/s]

Solved in 6 steps (5 tiles)
State(taken={6, 12, 25, 27, 28}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49})


In [181]:
def is_admissibile(h):
    for s in range(NUM_SETS):
        for t in range(NUM_SETS):
            if s != t:
                for _ in SETS[s] & ~SETS[t]:
                    h_s = h(State({s}, set(range(NUM_SETS)) - {s}))
                    h_t = h(State({t}, set(range(NUM_SETS)) - {t}))

                    cost_s_to_t = 1
                    
                    # Check the Triangle Inequality
                    if h_s > cost_s_to_t + h_t:
                        print("Heuristic is not admissible.")
                        return
    
    print("Heuristic is admissible.")

is_admissibile(h4)


Heuristic is not admissible.
